<div style="
    background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #5C7CFA 100%);
    padding: 40px;
    border-radius: 15px;
    margin: 20px 0;
    box-shadow: 0 8px 32px rgba(102, 126, 234, 0.3);
">
    <h1 style="
        color: white;
        text-align: center;
        margin: 0;
        font-size: 36px;
        font-weight: 300;
        letter-spacing: 3px;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
    ">MODEL INTERPRETABILITY</h1>
    <p style="
        color: rgba(255,255,255,0.95);
        text-align: center;
        margin: 15px 0 0 0;
        font-size: 16px;
        font-weight: 300;
        letter-spacing: 1px;
    ">SHAP Analysis for Physics-Grounded Feature Understanding</p>
</div>

## Objectives

This notebook uses SHAP (SHapley Additive exPlanations) to interpret the trained XGBoost classifier and validate that its decisions align with astrophysical expectations. We investigate:

1. **Which features dominate classification decisions?** — Do physically meaningful features (Hα, Ca II, colour indices) rank among the most important, or does the model rely on observational artifacts?
2. **Does the model prioritize physical features vs. observational artifacts?** — We compare SHAP importance of spectroscopic features (line profiles, equivalent widths) against metadata (SNR, seeing, fiber ID).
3. **Are there unexpected feature interactions revealing new physics?** — SHAP dependence plots can expose non-linear interactions that simple correlation analysis cannot detect.

**Validation Context:**  
SHAP values provide a **model-agnostic** decomposition of each prediction into per-feature contributions, grounded in cooperative game theory. For tree-based models (XGBoost), exact SHAP values can be computed via `TreeExplainer` in polynomial time, making this analysis both rigorous and tractable.

## Theoretical Background

SHAP (Lundberg & Lee, 2017) assigns each feature a **Shapley value** — the average marginal contribution across all possible feature coalitions. Unlike global feature importance (e.g., Gini importance), SHAP values are **local** (per-prediction) and **additive** (they sum to the model output minus the expected value). This makes SHAP the gold standard for post-hoc interpretability: it reveals not just *which* features matter, but *how* and *for which classes*.

For stellar classification, we expect spectroscopic line features (Hα FWHM, Ca II K prominence, Mg b equivalent width) and photometric colours (g−r, Blue/Red index) to dominate. If the model instead relies heavily on metadata (fiber ID, Julian date, seeing), this signals potential data leakage or confounding.

## Key References

- **Lundberg, S. M. & Lee, S.-I. (2017)** — *"A Unified Approach to Interpreting Model Predictions"*, NeurIPS 2017.
- **Lundberg, S. M. et al. (2020)** — *"From Local Explanations to Global Understanding with Explainable AI for Trees"*, Nature Machine Intelligence, 2, 56–67.
- **Luo, A.-L. et al. (2015)** — *"The First Data Release of the LAMOST Regular Survey"*, RAA, 15, 1095.

<a id="toc"></a>

## 📑 Table of Contents

<div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px; margin: 20px 0;">

<!-- Setup -->
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3); height: 120px; display: flex; flex-direction: column; justify-content: center;">
    <a href="#setup" style="color: white; text-decoration: none;">
        <strong style="font-size: 16px; display: block; margin-bottom: 8px;">⚙️ Section 1: Setup & Model Loading</strong>
        <span style="font-size: 12px; opacity: 0.9; line-height: 1.4;">Imports, model loading, SHAP explainer creation</span>
    </a>
</div>

<!-- Global Importance -->
<div style="background: linear-gradient(135deg, #764ba2 0%, #667eea 100%); padding: 20px; border-radius: 10px; box-shadow: 0 4px 15px rgba(118, 75, 162, 0.3); height: 120px; display: flex; flex-direction: column; justify-content: center;">
    <a href="#global" style="color: white; text-decoration: none;">
        <strong style="font-size: 16px; display: block; margin-bottom: 8px;">📊 Section 2: Global Feature Importance</strong>
        <span style="font-size: 12px; opacity: 0.9; line-height: 1.4;">Mean |SHAP| ranking, summary plots</span>
    </a>
</div>

<!-- Class-Specific -->
<div style="background: linear-gradient(135deg, #5C7CFA 0%, #764ba2 100%); padding: 20px; border-radius: 10px; box-shadow: 0 4px 15px rgba(92, 124, 250, 0.3); height: 120px; display: flex; flex-direction: column; justify-content: center;">
    <a href="#classwise" style="color: white; text-decoration: none;">
        <strong style="font-size: 16px; display: block; margin-bottom: 8px;">🔬 Section 3: Class-Specific SHAP</strong>
        <span style="font-size: 12px; opacity: 0.9; line-height: 1.4;">Per-class feature drivers & beeswarm plots</span>
    </a>
</div>

<!-- Interactions -->
<div style="background: linear-gradient(135deg, #667eea 0%, #5C7CFA 100%); padding: 20px; border-radius: 10px; box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3); height: 120px; display: flex; flex-direction: column; justify-content: center;">
    <a href="#interactions" style="color: white; text-decoration: none;">
        <strong style="font-size: 16px; display: block; margin-bottom: 8px;">🔗 Section 4: Feature Interactions</strong>
        <span style="font-size: 12px; opacity: 0.9; line-height: 1.4;">SHAP dependence plots & interaction effects</span>
    </a>
</div>

<!-- Discovery -->
<div style="background: linear-gradient(135deg, #764ba2 0%, #5C7CFA 100%); padding: 20px; border-radius: 10px; box-shadow: 0 4px 15px rgba(118, 75, 162, 0.3); height: 120px; display: flex; flex-direction: column; justify-content: center;">
    <a href="#discovery" style="color: white; text-decoration: none;">
        <strong style="font-size: 16px; display: block; margin-bottom: 8px;">💡 Section 5: Discovery Synthesis</strong>
        <span style="font-size: 12px; opacity: 0.9; line-height: 1.4;">Physical vs. artifact features, key findings</span>
    </a>
</div>

<!-- Summary -->
<div style="background: linear-gradient(135deg, #5C7CFA 0%, #667eea 100%); padding: 20px; border-radius: 10px; box-shadow: 0 4px 15px rgba(92, 124, 250, 0.3); height: 120px; display: flex; flex-direction: column; justify-content: center;">
    <a href="#summary" style="color: white; text-decoration: none;">
        <strong style="font-size: 16px; display: block; margin-bottom: 8px;">📋 Section 6: Summary & Composite Figure</strong>
        <span style="font-size: 12px; opacity: 0.9; line-height: 1.4;">Publication-quality figure & conclusions</span>
    </a>
</div>

</div>

<div style="text-align: center; margin: 30px 0; padding: 15px; background: linear-gradient(135deg, rgba(102, 126, 234, 0.1), rgba(118, 75, 162, 0.1)); border-radius: 10px; border: 1px solid rgba(102, 126, 234, 0.3);">
    <p style="margin: 0; color: #667eea; font-size: 13px;">💡 <strong>Quick Navigation:</strong> Click any section above to jump directly</p>
</div>

<div style="margin: 40px 0;">
    <div style="height: 3px; background: linear-gradient(90deg, transparent, #667eea, #764ba2, #5C7CFA, transparent); border-radius: 3px;"></div>
</div>

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px 30px; border-radius: 12px; margin: 30px 0 20px 0; box-shadow: 0 8px 25px rgba(102, 126, 234, 0.3); border: 2px solid rgba(255, 255, 255, 0.1);">
    <h2 style="color: white; margin: 0; text-align: center; font-size: 28px; font-weight: 300; letter-spacing: 2px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">⚙️ SETUP & MODEL LOADING</h2>
</div>

<a id="setup"></a>

<div style="text-align: right; margin: 10px 0;">
    <a href="#toc" style="color: #667eea; text-decoration: none; font-size: 12px;">⬆️ Back to TOC</a>
</div>

In [ ]:
# ──────────────────────────────────────────────
# Setup: imports, style, reproducibility
# ──────────────────────────────────────────────

from __future__ import annotations

import json
import sys
import warnings
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="shap")

# ── Style ──
plt.style.use("dark_background")
plt.rcParams.update({
    "figure.facecolor": "#1a1a2e",
    "axes.facecolor": "#16213e",
    "axes.edgecolor": "#e94560",
    "axes.labelcolor": "white",
    "text.color": "white",
    "xtick.color": "white",
    "ytick.color": "white",
    "grid.color": "#333333",
    "grid.alpha": 0.3,
    "font.size": 11,
    "figure.dpi": 100,
})
sns.set_palette("colorblind")

# ── Paths ──
ROOT = Path("..").resolve()
MODEL_DIR = ROOT / "data" / "models"
FEATURES_DIR = ROOT / "data" / "processed"
FIG_DIR = ROOT / "logs" / "shap"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# Add project to path
if str(ROOT / "src") not in sys.path:
    sys.path.insert(0, str(ROOT / "src"))

print(f"Project root : {ROOT}")
print(f"Figures saved to : {FIG_DIR}")
print(f"Models directory : {MODEL_DIR}")
print(f"Features directory: {FEATURES_DIR}")

In [ ]:
# ──────────────────────────────────────────────
# Load trained model + features
# ──────────────────────────────────────────────

from pipeline.classifier import SpectralClassifier

# ── Find the most recent XGBoost model ──
xgb_pkls = sorted(MODEL_DIR.glob("spectral_classifier_xgboost_*.pkl"))
if not xgb_pkls:
    raise FileNotFoundError("No XGBoost .pkl found in data/models/")

model_path = xgb_pkls[-1]  # most recent by timestamp
meta_path = model_path.with_suffix("").with_suffix("") if "_meta" in model_path.stem else None
meta_json = model_path.parent / (model_path.stem + "_meta.json")

print(f"Loading model: {model_path.name}")
clf: SpectralClassifier = SpectralClassifier.load_model(str(model_path))

# Read metadata
if meta_json.exists():
    with open(meta_json, "r", encoding="utf-8") as f:
        meta = json.load(f)
    print(f"  Model type     : {meta.get('model_type', '?')}")
    print(f"  Target         : {meta.get('prediction_target', '?')}")
    print(f"  Class labels   : {meta.get('class_labels', [])}")
    print(f"  Trained on     : {meta.get('trained_on_file', '?')}")
    print(f"  Selected feats : {len(meta.get('selected_features_', []))}")
else:
    meta = {}
    print("  ⚠ No metadata JSON found.")

# ── Load features dataset ──
feat_csvs = sorted(FEATURES_DIR.glob("features_*.csv"))
if not feat_csvs:
    raise FileNotFoundError("No features CSV found in data/processed/")

# Use the file the model was trained on, or the most recent
trained_file = meta.get("trained_on_file", "")
feat_path = None
for fp in feat_csvs:
    if fp.name == trained_file:
        feat_path = fp
        break
if feat_path is None:
    feat_path = feat_csvs[-1]
    print(f"  ⚠ Trained file '{trained_file}' not found, using latest: {feat_path.name}")

print(f"\nLoading features: {feat_path.name}")
df_feat = pd.read_csv(feat_path)
print(f"  Shape: {df_feat.shape}")

# ── Identify feature columns & label ──
# The pipeline was fitted on ALL candidate features (prep → fs → clf).
# We must feed the full set to prep; feature selection is handled by the fs step.
selected_feats = meta.get("selected_features_", []) or getattr(clf, "selected_features_", None) or []
all_candidate_feats = meta.get("feature_names_used", []) or getattr(clf, "feature_names_used", [])

# IMPORTANT: use all candidate features (pre-feature-selection) so that
# prep.transform(X) receives every column the ColumnTransformer expects.
use_feats = all_candidate_feats if all_candidate_feats else selected_feats

# Filter to columns actually present in the CSV
available = [c for c in use_feats if c in df_feat.columns]
missing = [c for c in use_feats if c not in df_feat.columns]
if missing:
    print(f"  ⚠ {len(missing)} features missing from CSV: {missing[:5]}...")
print(f"  Using {len(available)}/{len(use_feats)} candidate features for SHAP analysis")

# Build target label column
target = meta.get("prediction_target", "main_class")
if target == "main_class":
    df_feat["label"] = df_feat["subclass"].astype(str).str[0]
elif "label" not in df_feat.columns:
    df_feat["label"] = df_feat["subclass"]

# Drop rows with missing label or features
mask_valid = df_feat["label"].notna() & df_feat[available].notna().all(axis=1)
df_valid = df_feat.loc[mask_valid].copy()

X = df_valid[available].astype(np.float32)
y = df_valid["label"]

print(f"\n── Dataset Summary ──")
print(f"  Samples     : {len(X):,}")
print(f"  Features    : {X.shape[1]}")
print(f"  Classes     : {sorted(y.unique().tolist())}")
print(f"\n── Class Distribution ──")
print(y.value_counts().sort_index().to_string())

In [ ]:
# ──────────────────────────────────────────────
# Create SHAP TreeExplainer
# ──────────────────────────────────────────────

# Extract the underlying pipeline and booster
pipeline = clf.model_pipeline
pipe_inner = getattr(pipeline, "base_estimator", pipeline)  # unwrap calibration if present

# Get the preprocessor and classifier steps
if hasattr(pipe_inner, "named_steps"):
    prep = pipe_inner.named_steps.get("prep", None)
    fs = pipe_inner.named_steps.get("fs", None)
    estimator = pipe_inner.named_steps.get("clf", None)
    print(f"Pipeline steps: {list(pipe_inner.named_steps.keys())}")
else:
    prep, fs, estimator = None, None, pipe_inner
    print("Model is not a Pipeline — using directly.")

# Unwrap wrappers until we reach the tree-based estimator
# Possible chain: ThresholdTunedClassifier → CalibratedClassifierCV
#                 → _Float64ProbaWrapper → XGBClassifier
TREE_TYPES = {"XGBClassifier", "XGBRegressor", "LGBMClassifier",
              "LGBMRegressor", "GradientBoostingClassifier",
              "RandomForestClassifier", "DecisionTreeClassifier"}

MAX_UNWRAP = 10
for _i in range(MAX_UNWRAP):
    etype = type(estimator).__name__
    if etype in TREE_TYPES:
        break  # found the actual tree model

    unwrapped = False
    # Try common wrapper attributes in priority order
    for attr in ("base", "base_estimator", "estimator"):
        if hasattr(estimator, attr):
            inner = getattr(estimator, attr)
            # CalibratedClassifierCV special case: use fitted calibrators
            if etype == "CalibratedClassifierCV" and hasattr(estimator, "calibrated_classifiers_"):
                inner = estimator.calibrated_classifiers_[0].estimator
            if inner is not None and inner is not estimator:
                print(f"  Unwrapped {etype} (.{attr}) → {type(inner).__name__}")
                estimator = inner
                unwrapped = True
                break
    if not unwrapped:
        print(f"  ⚠ Cannot unwrap {etype} further.")
        break

print(f"  Final estimator type: {type(estimator).__name__}")

# ── Transform features through the preprocessing pipeline ──
# We need the transformed data that the booster actually sees
if prep is not None:
    X_prep = pd.DataFrame(
        prep.transform(X),
        columns=prep.get_feature_names_out() if hasattr(prep, "get_feature_names_out") else available,
        index=X.index,
    )
    print(f"  After prep: {X_prep.shape}")
else:
    X_prep = X.copy()

# Apply feature selection if present
if fs is not None:
    X_transformed = pd.DataFrame(
        fs.transform(X_prep),
        index=X.index,
    )
    # Get selected feature names
    if hasattr(fs, "get_support"):
        selected_mask = fs.get_support()
        transformed_cols = X_prep.columns[selected_mask].tolist()
    else:
        transformed_cols = [f"f{i}" for i in range(X_transformed.shape[1])]
    X_transformed.columns = transformed_cols
    print(f"  After FS : {X_transformed.shape}")
else:
    X_transformed = X_prep.copy()

# ── Create SHAP explainer ──
# Workaround: SHAP < 0.46 can't parse XGBoost 2.x multiclass base_score vectors.
# We monkey-patch the SHAP loader's base_score parsing line to handle arrays.
import shap.explainers._tree as _stree

_orig_loader_init = _stree.XGBTreeModelLoader.__init__

def _patched_loader_init(self, xgb_model):
    """Handle multiclass base_score arrays from XGBoost 2.x."""
    import io
    import xgboost as _xgb
    from shap.explainers._tree import decode_ubjson_buffer

    _stree._check_xgboost_version(_xgb.__version__)
    model = xgb_model

    raw = xgb_model.save_raw(raw_format="ubj")
    with io.BytesIO(raw) as fd:
        jmodel = decode_ubjson_buffer(fd)

    learner = jmodel["learner"]
    lmp = learner["learner_model_param"]
    objective = learner["objective"]
    booster = learner["gradient_booster"]

    n_classes = max(int(lmp["num_class"]), 1)
    n_targets = max(int(lmp["num_target"]), 1)
    n_targets = max(n_targets, n_classes)

    if "gbtree" in booster and "model" not in booster:
        booster = booster["gbtree"]

    if booster["model"].get("iteration_indptr", None) is not None:
        iteration_indptr = np.asarray(booster["model"]["iteration_indptr"], dtype=np.int32)
        diff = np.diff(iteration_indptr)
    else:
        n_parallel_trees = int(booster["model"]["gbtree_model_param"]["num_parallel_tree"])
        diff = np.repeat(n_targets * n_parallel_trees, model.num_boosted_rounds())

    if np.any(diff != diff[0]):
        raise ValueError("vector-leaf is not yet supported:", diff)

    self.n_trees_per_iter = int(diff[0])
    self.n_targets = n_targets

    # ── PATCHED: handle multiclass base_score ──
    bs_raw = lmp["base_score"]
    if isinstance(bs_raw, str) and bs_raw.startswith("["):
        mc_scores = [float(x) for x in bs_raw.strip("[]").split(",")]
        base_score_val = float(np.mean(mc_scores))
    else:
        base_score_val = float(bs_raw)

    self.base_score = base_score_val
    assert self.n_trees_per_iter > 0

    self.name_obj = objective["name"]
    self.name_gbm = booster["name"]

    if self.name_obj in ("binary:logistic", "reg:logistic"):
        import scipy.special
        self.base_score = scipy.special.logit(base_score_val)
    elif self.name_obj in ("reg:gamma", "reg:tweedie", "count:poisson",
                            "survival:cox", "survival:aft"):
        self.base_score = np.log(base_score_val)
    else:
        self.base_score = base_score_val

    self.num_feature = int(lmp["num_feature"])
    self.num_class = int(lmp["num_class"])

    trees = booster["model"]["trees"]
    self.num_trees = len(trees)

    self.node_parents, self.node_cleft, self.node_cright = [], [], []
    self.node_sindex, self.children_default, self.sum_hess = [], [], []
    self.values, self.thresholds, self.threshold_types, self.features = [], [], [], []
    self.split_types, self.categories = [], []

    feature_types = model.feature_types
    if feature_types is not None:
        cat_idx = np.where(np.asarray(feature_types) == "c")[0]
        self.cat_feature_indices = cat_idx if len(cat_idx) > 0 else None
    else:
        self.cat_feature_indices = None

    def to_integers(data):
        assert isinstance(data, list)
        return np.asanyarray(data, dtype=np.uint8)

    for i in range(self.num_trees):
        tree = trees[i]
        self.node_parents.append(np.asarray(tree["parents"]))
        self.node_cleft.append(np.asarray(tree["left_children"], dtype=np.int32))
        self.node_cright.append(np.asarray(tree["right_children"], dtype=np.int32))
        self.node_sindex.append(np.asarray(tree["split_indices"], dtype=np.uint32))

        base_weight = np.asarray(tree["base_weights"], dtype=np.float32)
        if base_weight.size != self.node_cleft[-1].size:
            raise ValueError("vector-leaf is not yet supported.")

        default_left = to_integers(tree["default_left"])
        default_child = np.where(
            default_left == 1, self.node_cleft[-1], self.node_cright[-1]
        ).astype(np.int64)
        self.children_default.append(default_child)
        self.sum_hess.append(np.asarray(tree["sum_hessian"], dtype=np.float64))

        is_leaf = self.node_cleft[-1] == -1
        split_cond = np.asarray(tree["split_conditions"], dtype=np.float32)
        leaf_weight = np.where(is_leaf, split_cond, 0.0)
        thresholds = np.where(is_leaf, 0.0, split_cond)
        thresholds = np.where(is_leaf, 0.0, np.nextafter(thresholds, -np.float32(np.inf)))
        threshold_types = np.zeros_like(thresholds, dtype=np.int32)

        self.values.append(leaf_weight.reshape(leaf_weight.size, 1))
        self.thresholds.append(thresholds)
        self.threshold_types.append(threshold_types)
        self.features.append(np.asarray(tree["split_indices"], dtype=np.int64))

        split_types = to_integers(tree["split_type"])
        self.split_types.append(split_types)
        cat_segments = tree["categories_segments"]
        cat_sizes = tree["categories_sizes"]
        cat_nodes = tree["categories_nodes"]
        cats = tree["categories"]
        self.categories.append(
            self.parse_categories(cat_nodes, cat_segments, cat_sizes, cats, self.node_cleft[-1])
        )

# Apply the monkey-patch
_stree.XGBTreeModelLoader.__init__ = _patched_loader_init
print("✔ SHAP XGBTreeModelLoader patched for multiclass base_score")

# Now create the explainer
_booster = estimator.get_booster()
explainer = shap.TreeExplainer(_booster)
print(f"\n✔ SHAP TreeExplainer created for {type(estimator).__name__}")
print(f"  Input features: {X_transformed.shape[1]}")

# ── Compute SHAP values (subsample if dataset is large) ──
MAX_SAMPLES = 2000
if len(X_transformed) > MAX_SAMPLES:
    np.random.seed(42)
    idx_sample = np.random.choice(len(X_transformed), MAX_SAMPLES, replace=False)
    X_shap = X_transformed.iloc[idx_sample]
    y_shap = y.iloc[idx_sample]
    print(f"  Subsampled to {MAX_SAMPLES} for SHAP computation")
else:
    X_shap = X_transformed
    y_shap = y
    print(f"  Using all {len(X_shap)} samples")

print("\nComputing SHAP values (this may take a moment)...")
shap_values = explainer.shap_values(X_shap)

# For multiclass, shap_values can be:
#   - a list of arrays (one per class): each shape (n_samples, n_features)
#   - a 3D array (n_samples, n_features, n_classes) when using Booster directly
if isinstance(shap_values, list):
    n_classes = len(shap_values)
    print(f"  ✔ Multiclass SHAP (list): {n_classes} classes × {shap_values[0].shape} each")
elif isinstance(shap_values, np.ndarray) and shap_values.ndim == 3:
    # Convert 3D array → list of 2D arrays for consistent downstream handling
    n_classes = shap_values.shape[2]
    shap_values = [shap_values[:, :, c] for c in range(n_classes)]
    print(f"  ✔ Multiclass SHAP (3D→list): {n_classes} classes × {shap_values[0].shape} each")
else:
    # Wrap in list for consistent handling
    shap_values = [shap_values]
    n_classes = 1
    print(f"  ✔ Binary SHAP: {shap_values[0].shape}")

# Class labels from the model
class_labels = meta.get("class_labels", sorted(y.unique().tolist()))
print(f"  Class labels: {class_labels}")

<div style="margin: 40px 0;">
    <div style="height: 3px; background: linear-gradient(90deg, transparent, #667eea, #764ba2, #5C7CFA, transparent); border-radius: 3px;"></div>
</div>

<div style="background: linear-gradient(135deg, #764ba2 0%, #667eea 100%); padding: 20px 30px; border-radius: 12px; margin: 30px 0 20px 0; box-shadow: 0 8px 25px rgba(118, 75, 162, 0.3); border: 2px solid rgba(255, 255, 255, 0.1);">
    <h2 style="color: white; margin: 0; text-align: center; font-size: 28px; font-weight: 300; letter-spacing: 2px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">📊 GLOBAL FEATURE IMPORTANCE</h2>
</div>

<a id="global"></a>

<div style="text-align: right; margin: 10px 0;">
    <a href="#toc" style="color: #667eea; text-decoration: none; font-size: 12px;">⬆️ Back to TOC</a>
</div>

**Goal:** Identify which features have the largest average impact on model predictions across all classes and samples. We compute the mean absolute SHAP value per feature, aggregated across all classes.

In [ ]:
# ──────────────────────────────────────────────
# 2a. Mean |SHAP| bar chart — Top 15 features
# ──────────────────────────────────────────────

feature_names = X_shap.columns.tolist()

# Aggregate mean |SHAP| across all classes
mean_abs_shap = np.mean(
    [np.abs(sv).mean(axis=0) for sv in shap_values], axis=0
)

# Build importance DataFrame
df_importance = pd.DataFrame({
    "feature": feature_names,
    "mean_abs_shap": mean_abs_shap,
}).sort_values("mean_abs_shap", ascending=False).reset_index(drop=True)

# ── Categorise features by family for colour coding ──
def feature_family(name: str) -> str:
    n = name.lower()
    if "halpha" in n or "hα" in n or "ha_" in n or "h_alpha" in n:
        return "Hα (Balmer)"
    elif "hbeta" in n or "hβ" in n or "hb_" in n:
        return "Hβ (Balmer)"
    elif "caii" in n or "cahk" in n or "ca_" in n or "ca4227" in n or "cah2" in n or "cah3" in n:
        return "Ca II"
    elif "mg" in n or "mgb" in n:
        return "Mg b"
    elif "na_d" in n or "nad" in n:
        return "Na D"
    elif "color" in n or "colour" in n or "_gr" in n or "_ri" in n or "bluered" in n:
        return "Colour"
    elif "teff" in n or "logg" in n or "mh_" in n or "ag_" in n or "ebp" in n:
        return "Gaia Params"
    elif "snr" in n:
        return "SNR"
    elif "bp_" in n or "rp_" in n or "phot_" in n or "flux_" in n or "magnitude" in n or "m_bp" in n or "m_rp" in n or "m_g" in n:
        return "Photometry"
    elif "parallax" in n or "pm" in n or "radial" in n or "distance" in n or "v_tan" in n:
        return "Astrometry"
    elif "index_" in n or "dn4000" in n or "g4300" in n or "tio5" in n:
        return "Spectral Index"
    elif "ratio_" in n or "contrast" in n or "ew_" in n:
        return "Line Ratio"
    elif "redshift" in n:
        return "Redshift"
    elif "cont_" in n:
        return "Continuum"
    else:
        return "Other"

df_importance["family"] = df_importance["feature"].apply(feature_family)

# Colour palette for families
family_palette = {
    "Hα (Balmer)": "#e74c3c",
    "Hβ (Balmer)": "#ff6b6b",
    "Ca II": "#3498db",
    "Mg b": "#2ecc71",
    "Na D": "#f39c12",
    "Colour": "#9b59b6",
    "Gaia Params": "#1abc9c",
    "SNR": "#95a5a6",
    "Photometry": "#e67e22",
    "Astrometry": "#34495e",
    "Spectral Index": "#667eea",
    "Line Ratio": "#764ba2",
    "Redshift": "#d35400",
    "Continuum": "#27ae60",
    "Other": "#7f8c8d",
}

# ── Plot top 15 ──
TOP_N = 15
df_top = df_importance.head(TOP_N).iloc[::-1]  # reverse for horizontal bar

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(
    range(TOP_N), df_top["mean_abs_shap"],
    color=[family_palette.get(f, "#7f8c8d") for f in df_top["family"]],
    edgecolor="white", linewidth=0.3, alpha=0.9,
)
ax.set_yticks(range(TOP_N))
ax.set_yticklabels(df_top["feature"].tolist(), fontsize=10)
ax.set_xlabel("Mean |SHAP Value|", fontsize=12)
ax.set_title("Global Feature Importance (Mean |SHAP|) — Top 15", fontsize=14, pad=15)
ax.grid(axis="x", alpha=0.3)

# Legend for families present in top 15
families_shown = df_top["family"].unique()
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=family_palette.get(f, "#7f8c8d"), label=f) for f in families_shown]
ax.legend(handles=legend_elements, loc="lower right", fontsize=9, framealpha=0.8)

fig.tight_layout()
fig.savefig(FIG_DIR / "global_importance.png", dpi=150, bbox_inches="tight")
plt.show()

# ── Print top 5 ──
print("── Top 5 Features by Mean |SHAP| ──")
for i, row in df_importance.head(5).iterrows():
    print(f"  {i+1}. {row['feature']:40s}  |SHAP| = {row['mean_abs_shap']:.4f}  [{row['family']}]")

# ── Quick assessment ──
physical = {"Hα (Balmer)", "Hβ (Balmer)", "Ca II", "Mg b", "Na D", "Colour", "Spectral Index", "Line Ratio", "Continuum"}
n_physical_top15 = df_importance.head(TOP_N)["family"].isin(physical).sum()
print(f"\n  Physical features in top {TOP_N}: {n_physical_top15}/{TOP_N}")
print(f"  → {'✔ Model relies primarily on physics' if n_physical_top15 >= TOP_N // 2 else '⚠ Check for potential data leakage'}")

In [ ]:
# ──────────────────────────────────────────────
# 2b. SHAP summary beeswarm plot (aggregated)
# ──────────────────────────────────────────────

# For the beeswarm, we average SHAP across classes (absolute)
if len(shap_values) > 1:
    # Average across classes for a single summary
    shap_mean = np.mean(np.abs(np.array(shap_values)), axis=0)
else:
    shap_mean = np.abs(shap_values[0])

fig, ax = plt.subplots(figsize=(12, 8))
shap.summary_plot(
    shap_values if len(shap_values) > 1 else shap_values[0],
    X_shap,
    plot_type="bar",
    max_display=TOP_N,
    show=False,
    class_names=class_labels if len(shap_values) > 1 else None,
)
plt.title("SHAP Feature Importance by Class", fontsize=14, pad=15)
plt.tight_layout()
plt.savefig(FIG_DIR / "summary_bar.png", dpi=150, bbox_inches="tight")
plt.show()

print("✔ Summary bar plot saved")

<div style="margin: 40px 0;">
    <div style="height: 3px; background: linear-gradient(90deg, transparent, #667eea, #764ba2, #5C7CFA, transparent); border-radius: 3px;"></div>
</div>

<div style="background: linear-gradient(135deg, #5C7CFA 0%, #764ba2 100%); padding: 20px 30px; border-radius: 12px; margin: 30px 0 20px 0; box-shadow: 0 8px 25px rgba(92, 124, 250, 0.3); border: 2px solid rgba(255, 255, 255, 0.1);">
    <h2 style="color: white; margin: 0; text-align: center; font-size: 28px; font-weight: 300; letter-spacing: 2px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">🔬 CLASS-SPECIFIC SHAP ANALYSIS</h2>
</div>

<a id="classwise"></a>

<div style="text-align: right; margin: 10px 0;">
    <a href="#toc" style="color: #667eea; text-decoration: none; font-size: 12px;">⬆️ Back to TOC</a>
</div>

**Goal:** Understand which features drive predictions **for each spectral class individually**. A feature that is globally important may matter only for certain classes (e.g., Hα FWHM for A-type stars, TiO5 for M-type stars).

In [ ]:
# ──────────────────────────────────────────────
# 3a. Per-class top features (heatmap)
# ──────────────────────────────────────────────

# Build per-class mean |SHAP| matrix
class_importance = {}
for i, label in enumerate(class_labels):
    if i < len(shap_values):
        class_importance[label] = np.abs(shap_values[i]).mean(axis=0)

df_class_imp = pd.DataFrame(class_importance, index=feature_names)

# Select features that appear in top 10 of any class
top_per_class = set()
for col in df_class_imp.columns:
    top_per_class.update(df_class_imp[col].nlargest(10).index.tolist())

df_heatmap = df_class_imp.loc[list(top_per_class)].copy()
# Sort by overall importance
df_heatmap["_total"] = df_heatmap.sum(axis=1)
df_heatmap = df_heatmap.sort_values("_total", ascending=False).drop(columns="_total")

fig, ax = plt.subplots(figsize=(10, max(8, len(df_heatmap) * 0.35)))
sns.heatmap(
    df_heatmap.head(20),
    annot=True, fmt=".3f", cmap="magma",
    linewidths=0.5, linecolor="#333",
    xticklabels=df_heatmap.columns,
    yticklabels=df_heatmap.head(20).index,
    ax=ax,
)
ax.set_title("Mean |SHAP| by Feature × Class (Top 20)", fontsize=14, pad=15)
ax.set_xlabel("Spectral Class")
ax.set_ylabel("")
fig.tight_layout()
fig.savefig(FIG_DIR / "class_feature_heatmap.png", dpi=150, bbox_inches="tight")
plt.show()

print("── Per-Class Top 3 Features ──")
for label in class_labels:
    if label in df_class_imp.columns:
        top3 = df_class_imp[label].nlargest(3)
        feats = ", ".join([f"{n} ({v:.4f})" for n, v in top3.items()])
        print(f"  {label}: {feats}")

In [ ]:
# ──────────────────────────────────────────────
# 3b. Beeswarm plots per class
# ──────────────────────────────────────────────

n_cols = min(3, len(class_labels))
n_rows = (len(class_labels) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(7 * n_cols, 6 * n_rows))
if n_rows == 1 and n_cols == 1:
    axes = np.array([axes])
axes = axes.flatten()

for i, label in enumerate(class_labels):
    if i >= len(shap_values):
        break
    plt.sca(axes[i])
    shap.summary_plot(
        shap_values[i], X_shap,
        max_display=10,
        show=False,
        plot_size=None,
    )
    axes[i].set_title(f"Class: {label}", fontsize=13, pad=10)

# Hide unused axes
for j in range(len(class_labels), len(axes)):
    axes[j].set_visible(False)

fig.suptitle("SHAP Beeswarm Plots by Spectral Class", fontsize=16, y=1.01)
fig.tight_layout()
fig.savefig(FIG_DIR / "beeswarm_per_class.png", dpi=150, bbox_inches="tight")
plt.show()

print("✔ Per-class beeswarm plots saved")

**Interpretation — Class-Specific SHAP:**

The heatmap and beeswarm plots reveal **which features the model uses to distinguish each spectral class**. Key expectations:
- **A-type stars:** Hα features (FWHM, EW) should dominate due to maximum Balmer absorption.
- **K/M-type stars:** Ca II features, TiO5, and red colour indices should be most important.
- **F/G-type stars:** A mix of metallic lines (Mg b, Na D) and colour features.

If non-physical features (SNR, fiber ID, Julian date) appear in the top ranks for any class, this warrants investigation for potential confounding.

---

In [ ]:
# ──────────────────────────────────────────────
# 3c. Class-specific multi-panel figure (top 10 per class)
# ──────────────────────────────────────────────

# Colorblind-friendly palette for each class
CLASS_PALETTE = {
    "A": "#4477AA",   # blue
    "F": "#66CCEE",   # cyan
    "G": "#228833",   # green
    "K": "#CCBB44",   # yellow
    "M": "#EE6677",   # red
    "s": "#AA3377",   # purple
}

# Helper: clean feature names for display
def clean_feat(name: str) -> str:
    """Remove common prefixes for cleaner axis labels."""
    for prefix in ("feature_", "feat_", "f_"):
        if name.lower().startswith(prefix):
            name = name[len(prefix):]
    return name

# Filter to stellar classes only (exclude 's' if present)
stellar_classes = [c for c in class_labels if c != "s"]
n_classes = len(stellar_classes)

fig, axes = plt.subplots(1, n_classes, figsize=(4.2 * n_classes, 6), sharey=False)
if n_classes == 1:
    axes = [axes]

TOP_K = 10
for idx, label in enumerate(stellar_classes):
    ax = axes[idx]
    cls_idx = class_labels.index(label)
    if cls_idx >= len(shap_values):
        ax.set_visible(False)
        continue

    # Top-K features for this class
    mean_shap_cls = np.abs(shap_values[cls_idx]).mean(axis=0)
    top_indices = np.argsort(mean_shap_cls)[-TOP_K:][::-1]
    top_vals = mean_shap_cls[top_indices]
    top_names = [clean_feat(feature_names[i]) for i in top_indices]

    # Horizontal bar chart — reversed for top-down ordering
    bars = ax.barh(
        range(TOP_K - 1, -1, -1), top_vals,
        color=CLASS_PALETTE.get(label, "#999999"),
        edgecolor="white", linewidth=0.3, alpha=0.9,
    )
    ax.set_yticks(range(TOP_K - 1, -1, -1))
    ax.set_yticklabels(top_names, fontsize=8)
    ax.set_xlabel("Mean |SHAP|", fontsize=9)
    ax.set_title(f"Class {label}", fontsize=13, fontweight="bold",
                 color=CLASS_PALETTE.get(label, "white"), pad=10)
    ax.grid(axis="x", alpha=0.2)
    ax.tick_params(axis="y", length=0)

fig.suptitle(
    "Top 10 Features per Spectral Class (SHAP)",
    fontsize=15, fontweight="bold", y=1.02,
)
fig.tight_layout()
fig.savefig(FIG_DIR / "class_specific_importance.png", dpi=150, bbox_inches="tight")
plt.show()

print("✔ Class-specific multi-panel figure saved → class_specific_importance.png")

In [ ]:
# ──────────────────────────────────────────────
# 3d. Cross-class comparison heatmap (top 20 global → per class)
# ──────────────────────────────────────────────

# Use top 20 globally-ranked features
top20_global = df_importance.head(20)["feature"].tolist()
df_cross = df_class_imp.loc[
    df_class_imp.index.isin(top20_global), stellar_classes
].copy()

# Sort rows by global importance (df_importance order)
df_cross = df_cross.reindex(top20_global)
df_cross = df_cross.dropna(how="all")

# Clean names for display
display_names = [clean_feat(f) for f in df_cross.index]

fig, ax = plt.subplots(figsize=(10, max(9, len(df_cross) * 0.42)))
sns.heatmap(
    df_cross.values,
    annot=True, fmt=".3f", cmap="viridis",
    linewidths=0.5, linecolor="#333",
    xticklabels=stellar_classes,
    yticklabels=display_names,
    ax=ax,
    cbar_kws={"label": "Mean |SHAP|", "shrink": 0.8},
)
ax.set_title(
    "Cross-Class Feature Importance (Top 20 Global Features)",
    fontsize=14, pad=15,
)
ax.set_xlabel("Spectral Class", fontsize=12)
ax.set_ylabel("")
ax.tick_params(axis="y", labelsize=9)

fig.tight_layout()
fig.savefig(FIG_DIR / "importance_heatmap.png", dpi=150, bbox_inches="tight")
plt.show()

# ── Per-class top 3 with physics interpretation ──
PHYSICS_HINTS = {
    "A": "Balmer lines dominate (Hα, Hβ FWHM/EW) — consistent with A-type physics.",
    "F": "Mix of Balmer + metallic lines expected (Ca K, Mg b begin to appear).",
    "G": "Metallic absorption lines (Na D, Mg b, Ca) + colour indices.",
    "K": "Ca II, Na D, molecular bands (TiO5) + red colour indices.",
    "M": "TiO/CaH molecular bands, very red colours, Ca II triplet.",
}

print("═" * 60)
print("  PER-CLASS TOP 3 FEATURES — PHYSICS CONTEXT")
print("═" * 60)
for label in stellar_classes:
    if label in df_class_imp.columns:
        top3 = df_class_imp[label].nlargest(3)
        feats = ", ".join([f"{clean_feat(n)} ({v:.4f})" for n, v in top3.items()])
        hint = PHYSICS_HINTS.get(label, "")
        print(f"\n  Class {label}: {feats}")
        print(f"    ↳ {hint}")
print()
print("✔ Cross-class heatmap saved → importance_heatmap.png")

<div style="margin: 40px 0;">
    <div style="height: 3px; background: linear-gradient(90deg, transparent, #667eea, #764ba2, #5C7CFA, transparent); border-radius: 3px;"></div>
</div>

<div style="background: linear-gradient(135deg, #667eea 0%, #5C7CFA 100%); padding: 20px 30px; border-radius: 12px; margin: 30px 0 20px 0; box-shadow: 0 8px 25px rgba(102, 126, 234, 0.3); border: 2px solid rgba(255, 255, 255, 0.1);">
    <h2 style="color: white; margin: 0; text-align: center; font-size: 28px; font-weight: 300; letter-spacing: 2px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">🔗 FEATURE INTERACTIONS</h2>
</div>

<a id="interactions"></a>

<div style="text-align: right; margin: 10px 0;">
    <a href="#toc" style="color: #667eea; text-decoration: none; font-size: 12px;">⬆️ Back to TOC</a>
</div>

**Goal:** Use SHAP dependence plots to visualise how individual feature values affect the model output, and detect **interaction effects** between features. A dependence plot shows feature value (x-axis) vs. SHAP value (y-axis), coloured by the strongest interacting feature.

In [ ]:
# ──────────────────────────────────────────────
# 4a. SHAP dependence plots for top features
# ──────────────────────────────────────────────

# Pick the top 4 globally important features
top_4_feats = df_importance.head(4)["feature"].tolist()

# Use class 0 SHAP values (or the single array) for dependence
sv_for_dep = shap_values[0]  # first class (usually A)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, feat in enumerate(top_4_feats):
    if feat not in X_shap.columns:
        continue
    feat_idx = X_shap.columns.tolist().index(feat)
    ax = axes[i]
    shap.dependence_plot(
        feat_idx, sv_for_dep, X_shap,
        interaction_index="auto",
        ax=ax, show=False,
    )
    ax.set_title(f"{feat}", fontsize=12, pad=10)
    ax.grid(alpha=0.2)

fig.suptitle(
    f"SHAP Dependence Plots (Class: {class_labels[0]})",
    fontsize=14, y=1.01,
)
fig.tight_layout()
fig.savefig(FIG_DIR / "dependence_top4.png", dpi=150, bbox_inches="tight")
plt.show()

print("✔ Dependence plots saved")

In [ ]:
# ──────────────────────────────────────────────
# 4b. Feature interaction matrix (SHAP-based)
# ──────────────────────────────────────────────

# Compute approximate interaction strengths using SHAP value correlations
# For each pair of features, measure how correlated their SHAP values are
top_10_feats = df_importance.head(10)["feature"].tolist()
top_10_idx = [feature_names.index(f) for f in top_10_feats if f in feature_names]

# Use absolute SHAP correlations across all classes as proxy for interaction
shap_concat = np.mean([np.abs(sv) for sv in shap_values], axis=0)
shap_top10 = shap_concat[:, top_10_idx]

corr_shap = np.corrcoef(shap_top10.T)
np.fill_diagonal(corr_shap, 0)  # remove self-correlation

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    pd.DataFrame(corr_shap, index=top_10_feats, columns=top_10_feats),
    annot=True, fmt=".2f", cmap="coolwarm", center=0,
    square=True, linewidths=0.5, linecolor="#333",
    vmin=-1, vmax=1, ax=ax,
)
ax.set_title("SHAP Value Correlation Matrix (Top 10 Features)", fontsize=14, pad=15)
fig.tight_layout()
fig.savefig(FIG_DIR / "shap_interaction_matrix.png", dpi=150, bbox_inches="tight")
plt.show()

# Identify strongest interactions
interaction_pairs = []
for i in range(len(top_10_feats)):
    for j in range(i + 1, len(top_10_feats)):
        interaction_pairs.append({
            "feature_1": top_10_feats[i],
            "feature_2": top_10_feats[j],
            "correlation": abs(corr_shap[i, j]),
        })

df_pairs = pd.DataFrame(interaction_pairs).sort_values("correlation", ascending=False)
print("── Strongest Feature Interactions (|SHAP correlation|) ──")
for _, row in df_pairs.head(5).iterrows():
    print(f"  {row['feature_1']} × {row['feature_2']}: {row['correlation']:.3f}")

<div style="margin: 40px 0;">
    <div style="height: 3px; background: linear-gradient(90deg, transparent, #667eea, #764ba2, #5C7CFA, transparent); border-radius: 3px;"></div>
</div>

<div style="background: linear-gradient(135deg, #764ba2 0%, #5C7CFA 100%); padding: 20px 30px; border-radius: 12px; margin: 30px 0 20px 0; box-shadow: 0 8px 25px rgba(118, 75, 162, 0.3); border: 2px solid rgba(255, 255, 255, 0.1);">
    <h2 style="color: white; margin: 0; text-align: center; font-size: 28px; font-weight: 300; letter-spacing: 2px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">💡 DISCOVERY SYNTHESIS</h2>
</div>

<a id="discovery"></a>

<div style="text-align: right; margin: 10px 0;">
    <a href="#toc" style="color: #667eea; text-decoration: none; font-size: 12px;">⬆️ Back to TOC</a>
</div>

**Goal:** Assess whether the model's feature reliance is physically justified. We categorise features as **physical** (spectroscopic lines, colour indices, spectral indices) vs. **observational** (SNR, seeing, fiber ID, metadata) and quantify how much SHAP importance falls into each category.

In [ ]:
# ──────────────────────────────────────────────
# 5a. Physical vs. Observational importance breakdown
# ──────────────────────────────────────────────

# Categorise all features
PHYSICAL_FAMILIES = {
    "Hα (Balmer)", "Hβ (Balmer)", "Ca II", "Mg b", "Na D",
    "Colour", "Spectral Index", "Line Ratio", "Continuum",
}
OBSERVATIONAL_FAMILIES = {"SNR", "Other"}
GAIA_FAMILIES = {"Gaia Params", "Photometry", "Astrometry", "Redshift"}

df_importance["category"] = df_importance["family"].apply(
    lambda f: "Physical (spectroscopic)" if f in PHYSICAL_FAMILIES
    else "Gaia / Photometric" if f in GAIA_FAMILIES
    else "Observational / Metadata"
)

# Aggregate SHAP by category
cat_shap = df_importance.groupby("category")["mean_abs_shap"].agg(["sum", "count", "mean"])
cat_shap["pct"] = cat_shap["sum"] / cat_shap["sum"].sum() * 100
cat_shap = cat_shap.sort_values("sum", ascending=False)

print("── SHAP Importance by Category ──")
print(cat_shap.to_string())

# ── Donut chart ──
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Left: pie by category
cat_colors = {
    "Physical (spectroscopic)": "#667eea",
    "Gaia / Photometric": "#764ba2",
    "Observational / Metadata": "#95a5a6",
}
colors = [cat_colors.get(c, "#7f8c8d") for c in cat_shap.index]

wedges, texts, autotexts = axes[0].pie(
    cat_shap["sum"],
    labels=cat_shap.index,
    autopct="%1.1f%%",
    colors=colors,
    startangle=90,
    pctdistance=0.75,
    wedgeprops=dict(width=0.4, edgecolor="white", linewidth=2),
)
for t in texts + autotexts:
    t.set_fontsize(10)
axes[0].set_title("SHAP Importance by Feature Category", fontsize=13, pad=15)

# Right: family breakdown for physical features only
df_phys = df_importance[df_importance["category"] == "Physical (spectroscopic)"]
family_shap = df_phys.groupby("family")["mean_abs_shap"].sum().sort_values(ascending=True)

axes[1].barh(
    range(len(family_shap)), family_shap.values,
    color=[family_palette.get(f, "#7f8c8d") for f in family_shap.index],
    edgecolor="white", linewidth=0.3,
)
axes[1].set_yticks(range(len(family_shap)))
axes[1].set_yticklabels(family_shap.index, fontsize=10)
axes[1].set_xlabel("Cumulative Mean |SHAP|", fontsize=11)
axes[1].set_title("Physical Feature Families", fontsize=13, pad=15)
axes[1].grid(axis="x", alpha=0.3)

fig.tight_layout()
fig.savefig(FIG_DIR / "physical_vs_observational.png", dpi=150, bbox_inches="tight")
plt.show()

# ── Interpretation ──
phys_pct = cat_shap.loc["Physical (spectroscopic)", "pct"] if "Physical (spectroscopic)" in cat_shap.index else 0
gaia_pct = cat_shap.loc["Gaia / Photometric", "pct"] if "Gaia / Photometric" in cat_shap.index else 0
obs_pct = cat_shap.loc["Observational / Metadata", "pct"] if "Observational / Metadata" in cat_shap.index else 0

print(f"\n── Interpretation ──")
print(f"  Physical (spectroscopic) features account for {phys_pct:.1f}% of total SHAP importance.")
print(f"  Gaia / Photometric features account for {gaia_pct:.1f}%.")
print(f"  Observational metadata accounts for {obs_pct:.1f}%.")
if obs_pct > 20:
    print("  ⚠ Observational features contribute >20% — investigate potential data leakage.")
else:
    print("  ✔ Model decisions are primarily grounded in physical features.")

In [ ]:
# ──────────────────────────────────────────────
# 5b. Feature importance: SHAP vs. XGBoost native
# ──────────────────────────────────────────────

# Get XGBoost native feature importance for comparison
native_imp = {}
if hasattr(estimator, "feature_importances_"):
    native_imp_vals = estimator.feature_importances_
    native_feat_names = X_transformed.columns.tolist()
    for name, val in zip(native_feat_names, native_imp_vals):
        native_imp[name] = val

if native_imp:
    df_compare = df_importance.head(20).copy()
    df_compare["native_importance"] = df_compare["feature"].map(native_imp).fillna(0)
    df_compare["shap_rank"] = range(1, len(df_compare) + 1)
    df_compare["native_rank"] = df_compare["native_importance"].rank(ascending=False).astype(int)

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.scatter(
        df_compare["shap_rank"], df_compare["native_rank"],
        s=80, c=[family_palette.get(f, "#7f8c8d") for f in df_compare["family"]],
        edgecolors="white", linewidth=0.5, alpha=0.8, zorder=5,
    )
    # Add feature labels
    for _, row in df_compare.iterrows():
        ax.annotate(
            row["feature"][:20],
            (row["shap_rank"], row["native_rank"]),
            fontsize=7, alpha=0.8,
            xytext=(5, 5), textcoords="offset points",
        )
    # Perfect agreement line
    max_rank = len(df_compare)
    ax.plot([1, max_rank], [1, max_rank], "w--", alpha=0.3, label="Perfect agreement")
    ax.set_xlabel("SHAP Rank", fontsize=12)
    ax.set_ylabel("XGBoost Native Rank", fontsize=12)
    ax.set_title("Feature Ranking: SHAP vs. XGBoost Native Importance", fontsize=13, pad=15)
    ax.legend(fontsize=9)
    ax.grid(alpha=0.2)
    ax.invert_xaxis()
    ax.invert_yaxis()
    fig.tight_layout()
    fig.savefig(FIG_DIR / "shap_vs_native_rank.png", dpi=150, bbox_inches="tight")
    plt.show()

    # Spearman rank correlation
    from scipy import stats
    rho_rank, p_rank = stats.spearmanr(df_compare["shap_rank"], df_compare["native_rank"])
    print(f"Spearman ρ(SHAP rank, native rank) = {rho_rank:+.3f}  (p = {p_rank:.2e})")
    print(f"  → {'Strong agreement' if rho_rank > 0.7 else 'Moderate agreement' if rho_rank > 0.4 else 'Weak agreement'} between SHAP and native importance")
else:
    print("⚠ No native feature importances available — skipping comparison.")

In [ ]:
# ──────────────────────────────────────────────
# 5c. Discovery detection — expected vs. unexpected features
# ──────────────────────────────────────────────

# Define features expected from stellar physics literature
EXPECTED_PHYSICS = {
    "halpha", "hbeta", "h_alpha", "h_beta", "ca_ii", "ca_k", "ca_h",
    "mg_b", "na_d", "tio5", "cah", "balmer", "ew_", "fwhm_",
    "colour", "color", "bp_rp", "g_rp", "bp_g",
    "teff", "logg", "feh", "fe_h",
    "spectral_index", "line_ratio",
}

def is_expected(feature_name: str) -> bool:
    """Check if a feature matches known stellar physics features."""
    fname_low = feature_name.lower()
    return any(exp in fname_low for exp in EXPECTED_PHYSICS)

# Classify top 30 features
df_top30 = df_importance.head(30).copy()
df_top30["expected"] = df_top30["feature"].apply(is_expected)
df_top30["status"] = df_top30["expected"].map({True: "✔ Expected", False: "⚠ Unexpected"})

# Separate
df_expected = df_top30[df_top30["expected"]]
df_unexpected = df_top30[~df_top30["expected"]]

print("═" * 60)
print("  DISCOVERY DETECTION — TOP 30 FEATURES")
print("═" * 60)
print(f"\n  ✔ Expected features (physics-based): {len(df_expected)}/30")
print(f"  ⚠ Unexpected features:               {len(df_unexpected)}/30")

if len(df_unexpected) > 0:
    print("\n  ── Unexpected Features & Hypotheses ──")
    for _, row in df_unexpected.iterrows():
        fname = row["feature"]
        family = row["family"]
        shap_val = row["mean_abs_shap"]
        # Generate hypothesis based on family
        if family in ("SNR", "Other"):
            hypothesis = "Possible observational confound or data quality proxy"
        elif family in ("Astrometry", "Photometry"):
            hypothesis = "May encode distance/extinction as secondary physics proxy"
        elif family == "Gaia Params":
            hypothesis = "Gaia parameters may correlate with stellar type through selection effects"
        elif family == "Redshift":
            hypothesis = "Radial velocity may encode stellar population membership"
        else:
            hypothesis = "Feature may capture non-obvious spectral information — investigate"
        print(f"    • {clean_feat(fname):30s} |SHAP|={shap_val:.4f}  [{family}]")
        print(f"      ↳ Hypothesis: {hypothesis}")
else:
    print("\n  ✔ All top 30 features are consistent with known stellar physics.")

# Build discoveries DataFrame
discoveries = []
for _, row in df_unexpected.iterrows():
    discoveries.append({
        "feature": row["feature"],
        "family": row["family"],
        "mean_abs_shap": row["mean_abs_shap"],
        "global_rank": df_importance[df_importance["feature"] == row["feature"]].index[0] + 1,
        "type": "unexpected",
    })
for _, row in df_expected.iterrows():
    discoveries.append({
        "feature": row["feature"],
        "family": row["family"],
        "mean_abs_shap": row["mean_abs_shap"],
        "global_rank": df_importance[df_importance["feature"] == row["feature"]].index[0] + 1,
        "type": "expected",
    })
df_discoveries = pd.DataFrame(discoveries)
print(f"\n  Discovery DataFrame: {len(df_discoveries)} features classified.")

In [ ]:
# ──────────────────────────────────────────────
# 5d. Feature interaction values (SHAP interaction matrix)
# ──────────────────────────────────────────────

# Subsample for interaction computation (expensive: O(n * d^2))
INTERACTION_SAMPLES = min(1000, len(X_shap))
rng_int = np.random.RandomState(42)
int_idx = rng_int.choice(len(X_shap), INTERACTION_SAMPLES, replace=False)
X_int = X_shap.iloc[int_idx] if hasattr(X_shap, "iloc") else X_shap[int_idx]

print(f"Computing SHAP interaction values on {INTERACTION_SAMPLES} samples...")
print("  (this may take a few minutes for large feature sets)")

try:
    shap_interaction = explainer.shap_interaction_values(X_int)

    # For multiclass, average across classes
    if isinstance(shap_interaction, list):
        # Stack all classes and take mean of absolute values
        interaction_3d = np.mean([np.abs(sv) for sv in shap_interaction], axis=0)
    else:
        interaction_3d = np.abs(shap_interaction)

    # Average across samples → (n_features, n_features)
    interaction_matrix = interaction_3d.mean(axis=0)

    # Zero out diagonal (self-interactions are just main effects)
    np.fill_diagonal(interaction_matrix, 0)

    # Top 10 features by main-effect SHAP for the heatmap
    top10_idx = np.argsort(df_importance.head(10)["feature"].apply(
        lambda f: list(feature_names).index(f) if f in feature_names else -1
    ).values)
    top10_feat_idx = [
        list(feature_names).index(f)
        for f in df_importance.head(10)["feature"]
        if f in feature_names
    ]

    int_sub = interaction_matrix[np.ix_(top10_feat_idx, top10_feat_idx)]
    int_labels = [clean_feat(feature_names[i]) for i in top10_feat_idx]

    # ── Heatmap ──
    fig, ax = plt.subplots(figsize=(14, 10))
    sns.heatmap(
        int_sub,
        annot=True, fmt=".4f", cmap="magma",
        linewidths=0.5, linecolor="#333",
        xticklabels=int_labels,
        yticklabels=int_labels,
        ax=ax,
        cbar_kws={"label": "Mean |SHAP Interaction|", "shrink": 0.8},
    )
    ax.set_title(
        "Feature Interaction Values (Top 10 Features)",
        fontsize=14, pad=15,
    )
    ax.tick_params(axis="x", rotation=45)
    fig.tight_layout()
    fig.savefig(FIG_DIR / "feature_interactions.png", dpi=150, bbox_inches="tight")
    plt.show()

    # ── Top 5 interaction pairs ──
    n_feat = len(top10_feat_idx)
    pairs = []
    for i in range(n_feat):
        for j in range(i + 1, n_feat):
            pairs.append((int_labels[i], int_labels[j], int_sub[i, j]))
    pairs.sort(key=lambda x: x[2], reverse=True)

    print("\n── Top 5 Feature Interaction Pairs ──")
    for rank, (f1, f2, val) in enumerate(pairs[:5], 1):
        print(f"  {rank}. {f1} × {f2}  →  mean |interaction| = {val:.5f}")

    INTERACTION_COMPUTED = True
    print("\n✔ Interaction heatmap saved → feature_interactions.png")

except Exception as e:
    print(f"⚠ SHAP interaction values failed: {e}")
    print("  Falling back to correlation-based interaction proxy (already computed).")
    INTERACTION_COMPUTED = False
    interaction_matrix = None

In [ ]:
# ──────────────────────────────────────────────
# 5e. Artifact vs. Physics verdict & discoveries export
# ──────────────────────────────────────────────

# ── Artifact ratio analysis ──
n_total_feats = len(df_importance)
n_obs_feats = len(df_importance[df_importance["category"] == "Observational / Metadata"])
obs_in_top10 = len(df_importance.head(10)[df_importance.head(10)["category"] == "Observational / Metadata"])
obs_in_top20 = len(df_importance.head(20)[df_importance.head(20)["category"] == "Observational / Metadata"])

print("═" * 60)
print("  ARTIFACT vs. PHYSICS ANALYSIS")
print("═" * 60)
print(f"\n  Total features:                {n_total_feats}")
print(f"  Observational features:        {n_obs_feats}/{n_total_feats}")
print(f"  Observational in top 10:       {obs_in_top10}/10")
print(f"  Observational in top 20:       {obs_in_top20}/20")
print(f"  Observational SHAP share:      {obs_pct:.1f}%")

# ── Verdict ──
print("\n  ── VERDICT ──")
if obs_pct < 5:
    verdict = "EXCELLENT — Model is overwhelmingly physics-driven."
    verdict_emoji = "✔✔"
elif obs_pct < 15:
    verdict = "GOOD — Model is primarily physics-driven with minor observational contribution."
    verdict_emoji = "✔"
elif obs_pct < 25:
    verdict = "ACCEPTABLE — Non-negligible observational contribution; monitor for confounds."
    verdict_emoji = "⚠"
else:
    verdict = "CONCERNING — Significant observational reliance; investigate data leakage."
    verdict_emoji = "❌"

print(f"  {verdict_emoji} {verdict}")

if obs_in_top10 > 0:
    obs_top10_names = df_importance.head(10)[
        df_importance.head(10)["category"] == "Observational / Metadata"
    ]["feature"].tolist()
    print(f"\n  ⚠ Observational features in Top 10: {', '.join(obs_top10_names)}")
    print("    These features may encode indirect physical information or represent confounds.")
else:
    print("\n  ✔ No observational features in the top 10 — no immediate leakage concern.")

# ── Quantify physics vs artifacts per class ──
print("\n  ── Per-Class Physics Fraction ──")
for label in stellar_classes:
    if label in df_class_imp.columns:
        cls_imp = df_class_imp[label].sort_values(ascending=False)
        cls_top10 = cls_imp.head(10)
        cls_top10_phys = sum(
            1 for f in cls_top10.index
            if df_importance.loc[df_importance["feature"] == f, "category"].values[0]
            in ("Physical (spectroscopic)", "Gaia / Photometric")
            if f in df_importance["feature"].values
        )
        print(f"    Class {label}: {cls_top10_phys}/10 top features are physics-based")

# ── Save discoveries ──
df_discoveries["verdict"] = verdict
df_discoveries.to_csv(FIG_DIR / "discoveries.csv", index=False)

print(f"\n  ✔ Discoveries saved → discoveries.csv ({len(df_discoveries)} entries)")
print("═" * 60)

<div style="margin: 40px 0;">
    <div style="height: 3px; background: linear-gradient(90deg, transparent, #667eea, #764ba2, #5C7CFA, transparent); border-radius: 3px;"></div>
</div>

<div style="background: linear-gradient(135deg, #5C7CFA 0%, #667eea 100%); padding: 20px 30px; border-radius: 12px; margin: 30px 0 20px 0; box-shadow: 0 8px 25px rgba(92, 124, 250, 0.3); border: 2px solid rgba(255, 255, 255, 0.1);">
    <h2 style="color: white; margin: 0; text-align: center; font-size: 28px; font-weight: 300; letter-spacing: 2px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">📋 SUMMARY & COMPOSITE FIGURE</h2>
</div>

<a id="summary"></a>

<div style="text-align: right; margin: 10px 0;">
    <a href="#toc" style="color: #667eea; text-decoration: none; font-size: 12px;">⬆️ Back to TOC</a>
</div>

## Publication-Quality Composite Figure

A single "hero" figure summarising the SHAP analysis, suitable for README, paper, or presentations.

In [ ]:
# ──────────────────────────────────────────────
# FIGURE 2 — Enhanced Publication-Quality SHAP Composite
# ──────────────────────────────────────────────
# Layout:  (a) Global importance     | (b) Class-specific small multiples
#          (c) Interaction heatmap   | (d) Physics vs Artifacts summary

from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(20, 16), constrained_layout=False)
gs = GridSpec(2, 2, figure=fig, hspace=0.35, wspace=0.30,
              left=0.06, right=0.97, top=0.93, bottom=0.04)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# (a) Global importance bar chart — top 15
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ax_a = fig.add_subplot(gs[0, 0])
df_top_plot = df_importance.head(15).iloc[::-1]
bars_a = ax_a.barh(
    range(15), df_top_plot["mean_abs_shap"],
    color=[family_palette.get(f, "#7f8c8d") for f in df_top_plot["family"]],
    edgecolor="white", linewidth=0.3, alpha=0.9,
)
ax_a.set_yticks(range(15))
ax_a.set_yticklabels([clean_feat(f) for f in df_top_plot["feature"]], fontsize=8)
ax_a.set_xlabel("Mean |SHAP|", fontsize=10)
ax_a.set_title("(a) Global Feature Importance — Top 15", fontsize=12, pad=10)
ax_a.grid(axis="x", alpha=0.2)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# (b) Class-specific small multiples (A, F, G, K, M)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
gs_b = gs[0, 1].subgridspec(1, len(stellar_classes), wspace=0.35)
TOP_COMPOSITE = 5  # fewer features for compact display

for idx, label in enumerate(stellar_classes):
    ax_bi = fig.add_subplot(gs_b[0, idx])
    cls_idx = class_labels.index(label)
    if cls_idx < len(shap_values):
        mean_cls = np.abs(shap_values[cls_idx]).mean(axis=0)
        top_idx = np.argsort(mean_cls)[-TOP_COMPOSITE:][::-1]
        top_vals = mean_cls[top_idx]
        top_names = [clean_feat(feature_names[i]) for i in top_idx]

        ax_bi.barh(
            range(TOP_COMPOSITE - 1, -1, -1), top_vals,
            color=CLASS_PALETTE.get(label, "#999"),
            edgecolor="white", linewidth=0.2, alpha=0.9,
        )
        ax_bi.set_yticks(range(TOP_COMPOSITE - 1, -1, -1))
        ax_bi.set_yticklabels(top_names, fontsize=6)
        ax_bi.tick_params(axis="x", labelsize=6)
        ax_bi.set_title(label, fontsize=11, fontweight="bold",
                        color=CLASS_PALETTE.get(label, "white"))
        ax_bi.grid(axis="x", alpha=0.15)
    else:
        ax_bi.set_visible(False)

# Panel title
fig.text(0.75, 0.935, "(b) Class-Specific Top Features", ha="center",
         fontsize=12, fontweight="normal")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# (c) Feature interaction heatmap (top 10 × 10)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ax_c = fig.add_subplot(gs[1, 0])
if INTERACTION_COMPUTED and interaction_matrix is not None:
    sns.heatmap(
        int_sub,
        annot=True, fmt=".3f", cmap="magma",
        linewidths=0.3, linecolor="#333",
        xticklabels=int_labels,
        yticklabels=int_labels,
        ax=ax_c,
        annot_kws={"fontsize": 7},
        cbar_kws={"shrink": 0.7},
    )
    ax_c.tick_params(axis="x", rotation=45, labelsize=7)
    ax_c.tick_params(axis="y", labelsize=7)
    ax_c.set_title("(c) Feature Interactions (Top 10 × 10)", fontsize=12, pad=10)
else:
    # Fallback: use cross-class heatmap
    hm_data = df_heatmap.head(10)
    sns.heatmap(
        hm_data, annot=True, fmt=".3f", cmap="magma",
        linewidths=0.3, linecolor="#333", ax=ax_c,
        annot_kws={"fontsize": 7}, cbar_kws={"shrink": 0.7},
    )
    ax_c.set_ylabel("")
    ax_c.set_title("(c) Feature × Class Importance (Top 10)", fontsize=12, pad=10)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# (d) Summary table — Physics vs Artifacts
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ax_d = fig.add_subplot(gs[1, 1])
ax_d.axis("off")

# Build comprehensive table
table_data = [
    ["Features analysed", str(len(feature_names))],
    ["Samples analysed", f"{len(X_shap):,}"],
    ["Classes", ", ".join(class_labels)],
    ["Top global feature", clean_feat(df_importance.iloc[0]["feature"])],
    ["", ""],
    ["Physical (spectroscopic)", f"{phys_pct:.1f}%"],
    ["Gaia / Photometric", f"{gaia_pct:.1f}%"],
    ["Observational / Metadata", f"{obs_pct:.1f}%"],
    ["", ""],
    ["Obs. features in top 10", f"{obs_in_top10}/10"],
    ["Unexpected in top 30", f"{len(df_unexpected)}/30"],
    ["Verdict", verdict_emoji + " " + verdict.split("—")[0].strip()],
]

table = ax_d.table(
    cellText=table_data,
    colLabels=["Metric", "Value"],
    cellLoc="center",
    loc="center",
)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.0, 1.6)

# Style table
for j in range(2):
    cell = table[0, j]
    cell.set_facecolor("#2a2a2a")
    cell.set_text_props(color="white", fontweight="bold")
for i in range(1, len(table_data) + 1):
    for j in range(2):
        cell = table[i, j]
        # Highlight verdict row
        if table_data[i - 1][0] == "Verdict":
            cell.set_facecolor("#1e3a5f")
        elif table_data[i - 1][0] == "":
            cell.set_facecolor("#111111")
            cell.set_edgecolor("#111111")
        else:
            cell.set_facecolor("#1a1a1a")
        cell.set_text_props(color="white")

ax_d.set_title("(d) Physics vs. Artifacts Summary", fontsize=12, pad=15)

# ── Super-title ──
fig.suptitle(
    "FIGURE 2 — SHAP Interpretability: Physics-Based ML for Stellar Classification",
    fontsize=16, fontweight="bold", y=0.98,
)
fig.text(
    0.5, 0.005,
    "AstroSpectro — LAMOST DR5 Analysis  •  XGBoost / TreeSHAP",
    ha="center", fontsize=9, style="italic", alpha=0.6,
)

fig.savefig(
    FIG_DIR / "FIGURE_2_shap_composite.png",
    dpi=300, bbox_inches="tight", facecolor=fig.get_facecolor(),
)
plt.show()

print(f"✔ Enhanced hero figure saved → {(FIG_DIR / 'FIGURE_2_shap_composite.png').resolve()}")

In [ ]:
# ──────────────────────────────────────────────
# Final SHAP analysis report
# ──────────────────────────────────────────────

print("=" * 60)
print("        SHAP INTERPRETABILITY REPORT")
print("=" * 60)
print()
print(f"  Model             : {meta.get('model_type', 'XGBoost')}")
print(f"  Target            : {meta.get('prediction_target', '?')}")
print(f"  Features analysed : {len(feature_names)}")
print(f"  Samples analysed  : {len(X_shap):,}")
print(f"  Classes           : {class_labels}")
print()
print("── Global Top 5 ──")
for i, row in df_importance.head(5).iterrows():
    print(f"  {i+1}. {row['feature']:40s} |SHAP| = {row['mean_abs_shap']:.4f}  [{row['family']}]")
print()
print("── Category Breakdown ──")
print(f"  Physical (spectroscopic) : {phys_pct:.1f}%")
print(f"  Gaia / Photometric       : {gaia_pct:.1f}%")
print(f"  Observational / Metadata : {obs_pct:.1f}%")
print()
if obs_pct < 20:
    print("  ✔ Model relies primarily on physically meaningful features.")
    print("  ✔ No evidence of data leakage through observational metadata.")
else:
    print("  ⚠ Significant observational feature reliance — investigate further.")
print()
print("─" * 60)
print("  References:")
print("  • Lundberg & Lee (2017) — NeurIPS, SHAP framework")
print("  • Lundberg et al. (2020) — Nature Machine Intelligence")
print("  • Luo et al. (2015) — RAA 15, 1095 (LAMOST DR5)")
print("=" * 60)

# ── Save importance CSV ──
df_importance.to_csv(FIG_DIR / "shap_feature_importance.csv", index=False)
print(f"\n✔ Feature importance saved to {(FIG_DIR / 'shap_feature_importance.csv').resolve()}")

---

### End of Notebook

All SHAP outputs have been saved to `../logs/shap/`:

| File | Description |
|---|---|
| `global_importance.png` | Top 15 features by mean \|SHAP\|, colour-coded by family |
| `summary_bar.png` | SHAP library summary bar plot by class |
| `class_feature_heatmap.png` | Per-class mean \|SHAP\| heatmap |
| `beeswarm_per_class.png` | Beeswarm plots for each spectral class |
| `class_specific_importance.png` | Multi-panel top 10 per class (A/F/G/K/M) |
| `importance_heatmap.png` | Cross-class comparison heatmap (top 20 global → per class) |
| `dependence_top4.png` | SHAP dependence plots for top 4 features |
| `shap_interaction_matrix.png` | SHAP value correlation matrix (interactions) |
| `feature_interactions.png` | SHAP interaction values heatmap (top 10 × 10) |
| `physical_vs_observational.png` | Category breakdown: physical vs. observational |
| `shap_vs_native_rank.png` | SHAP rank vs. XGBoost native importance rank |
| `shap_feature_importance.csv` | Machine-readable importance table |
| `discoveries.csv` | Feature discovery classification (expected vs. unexpected) |
| `FIGURE_2_shap_composite.png` | Publication-quality composite (300 DPI) |

**← Previous:** Scientific Validation (notebook 03).